In [ ]:
import visualize
import datasets
import ldle
import numpy as np
import os
%matplotlib qt
import pickle

In [ ]:
save_dir_root = 'D:/pyLDLE/LLR/'

In [ ]:
ar = 1
RES = 100
sideLx = np.sqrt(ar)
sideLy = 1/sideLx
RESx = int(sideLx*RES+1)
RESy = int(sideLy*RES+1)
x = np.linspace(0, sideLx, RESx)
y = np.linspace(0, sideLy, RESy)
xv, yv = np.meshgrid(x, y)
xv = xv.flatten('F')[:,np.newaxis]
yv = yv.flatten('F')[:,np.newaxis]
X = np.concatenate([xv,yv], axis=1)
labelsMat = X
print('X.shape = ', X.shape)

In [ ]:
N = 100

In [ ]:
lmbda = []
phi = []
grad_phi = []
for i in range(int(np.sqrt(N))+1):
    for j in range(int(np.sqrt(N))+1):
        lmbda.append((i/sideLx)**2 + (j/sideLy)**2)
        phi.append(np.cos(i*np.pi*X[:,0]/sideLx)*np.cos(j*np.pi*X[:,1]/sideLy))
        grad_phi.append([-(i*np.pi/sideLx)*np.sin(i*np.pi*X[:,0]/sideLx)*np.cos(j*np.pi*X[:,1]/sideLy),
                         -(j*np.pi/sideLy)*np.cos(i*np.pi*X[:,0]/sideLx)*np.sin(j*np.pi*X[:,1]/sideLy)])

In [ ]:
lmbda = np.array(lmbda)
phi = np.array(phi)
grad_phi = np.array(grad_phi)
inds = np.argsort(lmbda)[1:N+1]
lmbda = lmbda[inds]
phi = phi[inds,:].T
grad_phi = grad_phi[inds,:,:].T
print(lmbda.shape)
print(phi.shape)
print(grad_phi.shape)

In [ ]:
def square(use_LLR = False):
    X, labelsMat, _ = datasets.Datasets().rectanglegrid(ar=1)
    if use_LLR:
        fpath = save_dir_root+'/square_w_LLR.dat'
    else:
        fpath = save_dir_root+'/square_wo_LLR.dat'
        
    if os.path.exists(fpath):
        with open(fpath, "rb") as f:
            X, labelsMat, ldle_obj = pickle.load(f)
    else:
        ldle_obj = ldle.LDLE(X=X, eta_min=10, max_iter0=20, use_LLR = use_LLR,
                             lmbda = lmbda, phi = phi, N = N,
                             vis = visualize.Visualize(save_dir_root+'/square_'+str(int(use_LLR))),
                             vis_y_options = {'cmap0':'summer',
                                              'cmap1':'jet',
                                              'labels':labelsMat[:,0]})
        ldle_obj.fit()
        ldle_obj.y_final_ltsa = ldle_obj.compute_final_global_embedding_ltsap_based()
        with open(fpath, "wb") as f:
            pickle.dump([X, labelsMat, ldle_obj], f)
    return X, labelsMat, ldle_obj

# Run one of the above examples

In [ ]:
X, labelsMat, ldle_obj_w_LLR = square(use_LLR = True)

In [ ]:
X, labelsMat, ldle_obj_wo_LLR = square(use_LLR = False)

# Visualizations

# Eigenvalues $(\lambda_i)_{i=1}^{N}$

In [ ]:
ldle_obj_w_LLR.vis.eigenvalues(ldle_obj_w_LLR.lmbda)

In [ ]:
ldle_obj_wo_LLR.vis.eigenvalues(ldle_obj_wo_LLR.lmbda)

# Visualize Eigenvectors $\phi_{i}$ given $i$

In [ ]:
ldle_obj_w_LLR.vis.eigenvector(X, ldle_obj_w_LLR.phi, 5) # total #eigenvectors = ldle.N

In [ ]:
ldle_obj_wo_LLR.vis.eigenvector(X, ldle_obj_wo_LLR.phi, 5) # total #eigenvectors = ldle.N

# Visualize gamma $\gamma_i$ given $i$

In [ ]:
ldle_obj_w_LLR.vis.gamma(X, ldle_obj_w_LLR.gamma, 5) # total #eigenvectors = ldle.N

In [ ]:
ldle_obj_wo_LLR.vis.gamma(X, ldle_obj_wo_LLR.gamma, 5) # total #eigenvectors = ldle.N

# Visualize error in estimation of $\widetilde{A}_{kij}$ when use_LLR = True vs False

In [ ]:
from matplotlib import pyplot as plt
i = 0
j = 1
s = 20

In [ ]:
Atildeij_true = np.sum(grad_phi[:,:,i]*grad_phi[:,:,j], 1)

In [ ]:
fig = plt.figure(figsize=(12,4))
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()
plt.subplot(121)
plt.scatter(X[:,0], X[:,1], s=s, c=ldle_obj_w_LLR.phi[:,i], cmap='jet')
plt.axis('image')
plt.colorbar()
plt.title('$\phi_{%d}$'%i)
plt.subplot(122)
plt.scatter(X[:,0], X[:,1], s=s, c=ldle_obj_w_LLR.phi[:,j], cmap='jet')
plt.axis('image')
plt.colorbar()
plt.title('$\phi_{%d}$'%j)
plt.savefig(save_dir_root+'/LLR_vs_Our_phi_'+str(i)+'_and_phi_'+str(j)+'.png', dpi=150) 

In [ ]:
fig = plt.figure(figsize=(15,8))
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()
plt.subplot(231)
plt.scatter(X[:,0], X[:,1], s=s, c=Atildeij_true, cmap='jet')
plt.axis('image')
plt.colorbar()
plt.title('True $\widetilde{A}_{:%d%d}$'%(i,j))
plt.subplot(232)
plt.scatter(X[:,0], X[:,1], s=s, c=ldle_obj_w_LLR.Atilde[:,i,j], cmap='jet')
plt.axis('image')
plt.colorbar()
plt.title('Estimate of $\widetilde{A}_{:%d%d}$ with LLR'%(i,j))
plt.subplot(233)
plt.scatter(X[:,0], X[:,1], s=s, c=ldle_obj_wo_LLR.Atilde[:,i,j], cmap='jet')
plt.axis('image')
plt.colorbar()
plt.title('Estimate of $\widetilde{A}_{:%d%d}$ with our method'%(i,j))
plt.subplot(235)
plt.scatter(X[:,0], X[:,1], s=s, c=np.abs(Atildeij_true - ldle_obj_w_LLR.Atilde[:,i,j]), cmap='jet')
plt.axis('image')
plt.colorbar()
plt.title('|True - LLR Estimate|')
plt.subplot(236)
plt.scatter(X[:,0], X[:,1], s=s, c=np.abs(Atildeij_true - ldle_obj_wo_LLR.Atilde[:,i,j]), cmap='jet')
plt.axis('image')
plt.colorbar()
plt.title('|True - Our Estimate|')
plt.savefig(save_dir_root+'/LLR_vs_Our_Atilde_'+str(i)+'_'+str(j)+'.png') 

# Visualize $\nabla\phi_i$ given $i$ (works only when use_LLR = True)

In [ ]:
ldle_obj_w_LLR.vis.grad_phi(X, ldle_obj_w_LLR.phi, ldle_obj_w_LLR.grad_phi, 0)

# Distortion of Local Parameterizations $\zeta_{kk}$

In [ ]:
ldle_obj_w_LLR.vis.distortion(X, ldle_obj_w_LLR.local_param.zeta, 'zeta')

In [ ]:
ldle_obj_wo_LLR.vis.distortion(X, ldle_obj_wo_LLR.local_param.zeta, 'zeta')

# Distribution of distortion $\zeta_{kk}$

In [ ]:
ldle_obj_w_LLR.vis.compare_local_high_low_distortion(ldle_obj_w_LLR.X, ldle_obj_w_LLR.Atilde,
                                                     ldle_obj_w_LLR.local_param.Psi_gamma,
                                                     ldle_obj_w_LLR.local_param.Psi_i,
                                                     ldle_obj_w_LLR.local_param.zeta, 'zeta',
                                                     figsize=(20,4))

In [ ]:
ldle_obj_wo_LLR.vis.compare_local_high_low_distortion(ldle_obj_wo_LLR.X, ldle_obj_wo_LLR.Atilde,
                                                     ldle_obj_wo_LLR.local_param.Psi_gamma,
                                                     ldle_obj_wo_LLR.local_param.Psi_i,
                                                     ldle_obj_wo_LLR.local_param.zeta, 'zeta',
                                                     figsize=(20,4))

# Chosen eigenvectors' indices' for local views $i_1, i_2, \ldots, i_d$

In [ ]:
ldle_obj_w_LLR.vis.chosen_eigevec_inds_for_local_views(ldle_obj_w_LLR.X, ldle_obj_w_LLR.local_param.Psi_i)

In [ ]:
ldle_obj_wo_LLR.vis.chosen_eigevec_inds_for_local_views(ldle_obj_wo_LLR.X, ldle_obj_wo_LLR.local_param.Psi_i)

# Visualize final global embedding GPA based

In [ ]:
ldle_obj_w_LLR.vis.global_embedding(ldle_obj_w_LLR.y_final, labelsMat[:,1], 'jet',
                                    ldle_obj_w_LLR.color_of_pts_on_tear_final, 'jet', 'LLR')

In [ ]:
ldle_obj_wo_LLR.vis.global_embedding(ldle_obj_wo_LLR.y_final, labelsMat[:,1], 'jet',
                                     ldle_obj_wo_LLR.color_of_pts_on_tear_final, 'jet', 'Final')